# LiDAR Urban Planning in VR

Process and visualize city-scale LiDAR data in virtual reality.

In [ ]:
# Install required packages
# !pip install laspy numpy immersivepoints
# For LAZ (compressed LAS) support:
# !pip install laszip

In [ ]:
import numpy as np
import laspy
import immersivepoints as ip

print("LiDAR VR Visualization Ready!")

## Download Sample LiDAR Data

We'll download a sample LAS file. For real projects, download from:
- USGS 3DEP: https://www.usgs.gov/3d-elevation-program
- OpenTopography: https://opentopography.org/
- Your local government's GIS portal

In [ ]:
import requests
import os

# Sample LiDAR file (small urban area)
sample_url = "https://github.com/ASPRSorg/LAS/raw/master/test/simple.las"
sample_file = "sample_lidar.las"

if not os.path.exists(sample_file):
    print(f"Downloading sample LiDAR data...")
    response = requests.get(sample_url)
    with open(sample_file, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {len(response.content)/1024:.1f} KB")
else:
    print(f"Using existing {sample_file}")

# Or use your own file:
# sample_file = "path/to/your/lidar_file.las"

## Load and Inspect LiDAR Data

In [ ]:
print(f"Loading {sample_file}...")
las = laspy.read(sample_file)

print(f"\nLiDAR File Information:")
print(f"  Points: {len(las.points):,}")
print(f"  Format: LAS {las.header.version}")
print(f"  Point format: {las.header.point_format}")
print(f"\nBounding Box:")
print(f"  X: [{las.header.mins[0]:.2f}, {las.header.maxs[0]:.2f}]")
print(f"  Y: [{las.header.mins[1]:.2f}, {las.header.maxs[1]:.2f}]")
print(f"  Z: [{las.header.mins[2]:.2f}, {las.header.maxs[2]:.2f}]")

# Extract coordinates
x = las.x
y = las.y
z = las.z

print(f"\nAvailable fields: {list(las.point_format.dimension_names)}")

## Subsample for VR Performance

LiDAR files often have millions of points. We'll subsample to 100K-500K for smooth VR.

In [ ]:
def subsample_lidar(x, y, z, target_points=200000):
    """
    Intelligently subsample LiDAR data.
    """
    n_points = len(x)
    
    if n_points <= target_points:
        print(f"No subsampling needed ({n_points} points)")
        return x, y, z, np.arange(n_points)
    
    # Calculate sampling ratio
    ratio = target_points / n_points
    print(f"Subsampling {n_points:,} points to ~{target_points:,} ({ratio*100:.1f}%)")
    
    # Random sampling
    indices = np.random.choice(n_points, size=target_points, replace=False)
    indices.sort()  # Keep some spatial coherence
    
    return x[indices], y[indices], z[indices], indices

x_sub, y_sub, z_sub, indices = subsample_lidar(x, y, z, target_points=200000)
print(f"Subsampled to {len(x_sub):,} points")

## Color Coding Options

### Option 1: Color by Height (Elevation)

In [ ]:
def color_by_height(z, colormap='rainbow'):
    """
    Color points by elevation.
    """
    z_norm = (z - z.min()) / (z.max() - z.min())
    
    if colormap == 'rainbow':
        # Blue (low) -> Green -> Yellow -> Red (high)
        hue = 0.7 - (z_norm * 0.7)  # 0.7 to 0.0
    elif colormap == 'terrain':
        # Blue (water) -> Green (land) -> Brown (mountains)
        hue = np.where(z_norm < 0.3, 0.6,  # Blue for low
               np.where(z_norm < 0.7, 0.33,  # Green for mid
                        0.08))  # Brown for high
    else:
        hue = z_norm
    
    return hue

hue_height = color_by_height(z_sub, colormap='rainbow')
print(f"Colored {len(hue_height):,} points by elevation")

### Option 2: Color by LiDAR Classification

In [ ]:
def color_by_classification(las, indices):
    """
    Color by LiDAR classification codes.
    Standard classes: 2=Ground, 3=Low vegetation, 4=Med vegetation,
                     5=High vegetation, 6=Building, 9=Water
    """
    try:
        classification = las.classification[indices]
        
        class_colors = {
            0: 0.5,   # Never classified - white
            1: 0.5,   # Unclassified - white
            2: 0.08,  # Ground - brown
            3: 0.33,  # Low vegetation - green
            4: 0.33,  # Medium vegetation - green
            5: 0.33,  # High vegetation - green
            6: 0.0,   # Building - red/gray
            7: 0.0,   # Noise - dark
            9: 0.6,   # Water - blue
            17: 0.7,  # Bridge - blue-cyan
        }
        
        hue = np.array([class_colors.get(c, 0.5) for c in classification])
        print(f"Classification classes found: {np.unique(classification)}")
        return hue
    
    except:
        print("No classification data available, using height instead")
        return color_by_height(z_sub)

hue_class = color_by_classification(las, indices)

### Option 3: Color by Intensity (Reflectivity)

In [ ]:
def color_by_intensity(las, indices):
    """
    Color by LiDAR return intensity.
    """
    try:
        intensity = las.intensity[indices]
        intensity_norm = (intensity - intensity.min()) / (intensity.max() - intensity.min())
        
        # Grayscale: 0.0 (red/dark) to 0.0 (bright)
        # Actually map to hue for visibility
        hue = intensity_norm * 0.5  # 0 to 0.5 (dark to light)
        
        print(f"Intensity range: {intensity.min()} to {intensity.max()}")
        return hue
    except:
        print("No intensity data available")
        return color_by_height(z_sub)

hue_intensity = color_by_intensity(las, indices)

## Prepare Point Cloud for VR

In [ ]:
# Choose your color scheme (uncomment one):
hue = hue_height       # Color by elevation
# hue = hue_class      # Color by classification
# hue = hue_intensity  # Color by intensity

# Center the data
x_centered = x_sub - x_sub.mean()
y_centered = y_sub - y_sub.mean()
z_centered = z_sub - z_sub.min()  # Ground at Z=0

# Scale for VR (LiDAR coordinates are often in meters)
# Scale down so 1 meter = 0.01 VR units (makes large scenes manageable)
scale = 0.01
x_vr = x_centered * scale
y_vr = y_centered * scale
z_vr = z_centered * scale

# Create XYZI array
points = np.column_stack([x_vr, y_vr, z_vr, hue]).astype(np.float32)

print(f"\nVR Point Cloud:")
print(f"  Points: {len(points):,}")
print(f"  VR Size: X=[{x_vr.min():.1f}, {x_vr.max():.1f}] "
      f"Y=[{y_vr.min():.1f}, {y_vr.max():.1f}] "
      f"Z=[{z_vr.min():.1f}, {z_vr.max():.1f}]")
print(f"  Real Size: ~{(x_sub.max()-x_sub.min()):.0f}m × {(y_sub.max()-y_sub.min()):.0f}m")

## Visualize in Jupyter

In [ ]:
ip.renderPoints(points, point_size=0.02, background_color=0x87CEEB, show_axes=False)

## Generate VR Link

In [ ]:
ip.showVR(points, point_size=0.02)

## Save for Upload

In [ ]:
output_file = "lidar_urban_scene.xyzi"
points.astype(np.float32).byteswap().tofile(output_file)

print(f"Saved to {output_file}")
print(f"File size: {len(points) * 16 / (1024*1024):.1f} MB")
print(f"\nUpload at: https://immersivepoints.com/upload.html")

## Tips for Large Datasets

**If your LAS file is huge (>100MB):**

1. **Spatial crop**: Only load points in your area of interest
```python
# Crop to bounding box
xmin, xmax = 500000, 501000  # Adjust to your coordinates
ymin, ymax = 4000000, 4001000
mask = (x > xmin) & (x < xmax) & (y > ymin) & (y < ymax)
x_crop = x[mask]
y_crop = y[mask]
z_crop = z[mask]
```

2. **Aggressive subsampling**: Keep only 1:100 or 1:200 points

3. **Ground removal**: Filter out ground points to focus on buildings
```python
# Keep only non-ground points
if hasattr(las, 'classification'):
    non_ground = las.classification != 2
    x = x[non_ground]
    y = y[non_ground]
    z = z[non_ground]
```